## Alternating Framework Complete Code

1)calculate1 is a function that executes all trades on the input data starting from the entry index to the exit index and returns the final capital after executing all the trades in this duration

In [ ]:
def calculate1(dataa,cap,entry_index,exit_index):
  current=dataa['signals'].iloc[entry_index]
  for i in range(entry_index+1,exit_index+1):
      if(current==1):
          if(dataa['signals'].iloc[i]==1):
              dataa['signals'].iloc[i]=0
          elif(dataa['signals'].iloc[i]==-1):
              current=0
      elif(current==-1):
          if(dataa['signals'].iloc[i]==-1):
              dataa['signals'].iloc[i]=0
          elif(dataa['signals'].iloc[i]==1):
              current=0
      else:
          if(dataa['signals'].iloc[i]==1):
              current=1
          elif(dataa['signals'].iloc[i]==-1):
              current=-1
  num_buys=0 #Total no. of buy signals
  num_sells=0 #Total no. of sell signals
  buys=[] #List containing index of each buy trade
  sells=[] #List containing index of each sell trade
  trades=[] #List containing index of all trades
  for i in range(entry_index,exit_index+1):
    if(dataa.loc[i,'signals']==1):
      num_buys=num_buys+1
      buys.append(i)
      trades.append(i)
    elif(dataa.loc[i,'signals']==-1):
      num_sells=num_sells+1
      sells.append(i)
      trades.append(i)
  if(num_buys!=num_sells):
    # changing the signal of the last day to close all trades.
    if(dataa['signals'].iloc[trades[num_buys+num_sells-1]]==1):
      dataa['signals'].iloc[exit_index]=-1
      trades.append(exit_index)
    else:
      dataa['signals'].iloc[exit_index]=1
      trades.append(exit_index)
  num_closetrades=max(num_buys,num_sells) #Total no. of close trades
  bal=cap
  position=0
  for i in range(entry_index,exit_index+1):
    if(cap>0):
      if dataa['signals'].iloc[i]==1 and position==0:
          no_of_stocks=int(cap/dataa['Close'].iloc[i])
          price=dataa['Close'].iloc[i]
          remain=cap-no_of_stocks*price
          index1=i
          type="long"
          position=1

      elif dataa['signals'].iloc[i]==-1 and position==1:
          returns=(((dataa['Close'].iloc[i]-price) / price)*100)
          cap=no_of_stocks*(dataa['Close'].iloc[i])+remain
          position=0
          index2=i
      elif dataa['signals'].iloc[i]==-1 and position==0:
          no_of_stocks=int(cap / dataa['Close'].iloc[i])
          price=dataa['Close'].iloc[i]
          position=-1
          type="short"
          index1=i

      elif dataa['signals'].iloc[i]==1 and position==-1:
          returns=(((price-dataa['Close'].iloc[i]) / price)*100)
          cap=bal+no_of_stocks*(price-dataa['Close'].iloc[i])
          position=0
          index2=i
    else:
      print("Capital Wiped-OFF")
  return cap

2)calculate2 is a function that executes only one trade on an input data starting from the input index and returns a TradeWise DataFrame for that single trade

In [ ]:
def calculate2(dataa,cap,entry_index,num):
  bal=cap
  position=0
  index1=0
  index2=0
  returns=0
  type="none"
  no_of_stocks=0
  remain=0
  price=0
  for i in range(entry_index,len(dataa)):
    if(cap>0):
      if dataa['signals'].iloc[i]==1 and position==0:
          no_of_stocks=int(cap/dataa['Close'].iloc[i])
          price=dataa['Close'].iloc[i]
          remain=cap-no_of_stocks*price
          index1=i
          type="long"
          position=1

      elif dataa['signals'].iloc[i]==-1 and position==1:
          returns=(((dataa['Close'].iloc[i]-price) / price)*100)
          cap=no_of_stocks*(dataa['Close'].iloc[i])+remain
          position=0
          index2=i
          break
      elif dataa['signals'].iloc[i]==-1 and position==0:
          no_of_stocks=int(cap / dataa['Close'].iloc[i])
          price=dataa['Close'].iloc[i]
          position=-1
          type="short"
          index1=i

      elif dataa['signals'].iloc[i]==1 and position==-1:
          returns=(((price-dataa['Close'].iloc[i]) / price)*100)
          cap=bal+no_of_stocks*(price-dataa['Close'].iloc[i])
          position=0
          index2=i
          break
    else:
      print("Capital Wiped-OFF")
  if(no_of_stocks>0):
    drawdown=[]
    portfolio_list=[]
    max1=bal
    min1=bal
    #print("1,2",index1,index2)
    for j in range(index1,index2):
      if(dataa['signals'].iloc[index1]==1):
        portfolio_list.append((no_of_stocks*dataa['Close'].iloc[j])+remain)
      else:
        portfolio_list.append(bal+(no_of_stocks*(price-dataa['Close'].iloc[j])))
    if(len(portfolio_list)>0):
      portfolio_list.append(portfolio_list[len(portfolio_list)-1])
    #print(portfolio_list)
    for i in range(len(portfolio_list)):
      if (portfolio_list[i]>max1):
        max1=portfolio_list[i]
      if (portfolio_list[i]<min1):
        min1=portfolio_list[i]
      drawdown.append(((max1-portfolio_list[i])/portfolio_list[i])*100)
    if(len(drawdown)>0):
      max__drawdown=(max(drawdown))
    else:
      max__drawdown=0
    max__dip=((((bal)-min1)/(bal))*100)
    twd=pd.DataFrame(columns=['Strategy','Entry Index', 'Exit Index','Trade Duration', 'Returns for Trade in %', 'Type of Trade', 'Max Drawdown for Trade', 'Max Dip for Trade', 'No. of stocks Traded','Capital'])
    new_row = {'Strategy':num,'Entry Index':index1, 'Exit Index':index2,'Trade Duration':index2-index1,'Returns for Trade in %':returns, 'Type of Trade':type, 'Max Drawdown for Trade':max__drawdown, 'Max Dip for Trade':max__dip, 'No. of stocks Traded':no_of_stocks,'Capital':cap}
    twd.loc[len(twd)]=new_row
    return twd
  else:
    twd=pd.DataFrame(columns=['Strategy','Entry Index', 'Exit Index','Trade Duration', 'Returns for Trade in %', 'Type of Trade', 'Max Drawdown for Trade', 'Max Dip for Trade', 'No. of stocks Traded','Capital'])
    new_row = {'Strategy':num,'Entry Index':index1, 'Exit Index':len(data7),'Trade Duration':0,'Returns for Trade in %':0, 'Type of Trade':"none", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of stocks Traded':0,'Capital':cap}
    twd.loc[len(twd)]=new_row
    return twd

3)Complete Code of how I alternate between the Two Strategies.

In [ ]:
final_twd=pd.DataFrame(columns=['Strategy','Entry Index', 'Exit Index','Trade Duration', 'Returns for Trade in %', 'Type of Trade', 'Max Drawdown for Trade', 'Max Dip for Trade', 'No. of stocks Traded','Capital'])
#data2 contains signal of Bollinger Bands
#data7 contains signal of MACD
#data9 contains signal of RSI
z1=data7 # z1 contains signals corresponding to 1st Strategy
z2=data9 # z2 contains signals corresponding to 2nd Strategy
capitals=10000 #Initial Capital
i=0
strategy=1 # We take First Trade on Strategy 1
while(i<len(data7)):

  if(strategy==1):
    z1_copy = z1.copy()
    z3=calculate2(z1_copy,capitals,i,1)
    if(z3['No. of stocks Traded'].iloc[0]==0):
      break
    g=z3['Exit Index'].iloc[0]
    if(g==0):
      break;
    new_capital=z3['Capital'].iloc[0]
    final_twd=pd.concat([final_twd,z3],axis=0,ignore_index=True)
    z2_copy = z2.copy()
    p=calculate1(z2_copy,capitals,i,g)
    capitals=new_capital
    if(p>new_capital):
      strategy=2
      i=g
    else:
      i=g+1

  elif(strategy==2):
    z2_copy = z2.copy()
    z4=calculate2(z2_copy,capitals,i,2)
    if(z4['No. of stocks Traded'].iloc[0]==0):
      break
    f=z4['Exit Index'].iloc[0]
    if(f==0):
      break
    new_capital=z4['Capital'].iloc[0]
    final_twd=pd.concat([final_twd,z4],axis=0,ignore_index=True)
    z1_copy = z1.copy()
    p=calculate1(z1_copy,capitals,i,f)
    capitals=new_capital
    if(p>new_capital):
      strategy=1
      i=f
    else:
      i=f+1

4)Final TradeWise DataFrame of Alternate FrameWork Strategy.

In [ ]:
#Code

#Benchmark Return and Total Returns
balance=10000
num=int(balance/data2['Close'].iloc[0])
remain_=balance-num*data2['Close'].iloc[0]
br=((((num*data2['Close'].iloc[len(data2)-1])+remain_)-balance)/balance)*100
print("Benchmark Return is:",br,"%")
print("Total Returns of Strategy:",(((final_twd['Capital'].iloc[len(final_twd)-1])-balance)/balance)*100,"%")

#Gross Profit and Net Profit
gp=(final_twd['Capital'].iloc[len(final_twd)-1])-balance
print("Gross Profit is:",gp)
#Max Drawdown
maxDrawdown=max(final_twd['Max Drawdown for Trade'])
print("Maximun Drawdown is:",maxDrawdown)
#Avg. Drawdown
avgDrawdown=final_twd['Max Drawdown for Trade'].mean()
print("Average Drawdown is:",avgDrawdown)
#Max Dip
maxDip=max(final_twd['Max Dip for Trade'])
print("Maximun Dip is:",maxDip)
#Avg. Dip
avgDip=final_twd['Max Dip for Trade'].mean()
print("Average Dip is:",avgDip)
#Sharpe Ratio and Sortino Ratio
returns=final_twd['Returns for Trade in %']
rfr=[]
negative_returns=[]
entry=final_twd['Entry Index'].iloc[0]
exit=final_twd['Exit Index'].iloc[len(final_twd)-1]
for i in range(entry,exit+1):
  rfr.append(data3['Close'].iloc[i])
for i in range(len(final_twd)):
  if(final_twd['Returns for Trade in %'].iloc[i]<0):
    negative_returns.append(final_twd['Returns for Trade in %'].iloc[i])
mean_rfr=sum(rfr)/len(rfr)
mean_returns=returns.mean()
mean_neg_returns=sum(negative_returns)/len(negative_returns)
array=np.array(negative_returns)
std_neg_returns=np.std(array,ddof=1)
std_returns=returns.std()
y=((math.sqrt(252)))
sharpeRatio=y*((mean_returns-mean_rfr)/std_returns)
sortinoRatio=y*((mean_returns-mean_rfr)/std_neg_returns)
print("Calculated Sharpe Ratio of the strategy is:",sharpeRatio)
print("Calculated Sortino Ratio of the strategy is:",sortinoRatio)
#print("Calculated Sharpe Ratio of the strategy is:",sharpe_ratio(np.array(returns)))